In [2]:
%%capture
#@title
import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientdet" not in os.getcwd():
  !git clone --depth 1 https://github.com/rezabasiri/EfficientDetDFU
  os.chdir('EfficientDetDFU/automl/efficientdet')
  sys.path.append('.')
  !pip install -r requirements.txt
  !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
else:
  !git pull

In [ ]:
ckpt = '20200715_DFU2020_ckpt'
MODEL = 'efficientdet-d1'

def download(m):
  if m not in os.listdir():
    !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=11L4jYRgCBUzchWt35SOMc2v_dmgZdxat" > /tmp/intermezzo.html
    !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > FINAL_DOWNLOADED_FILENAME.zip
    !unzip FINAL_DOWNLOADED_FILENAME.zip
    !rm -rf FINAL_DOWNLOADED_FILENAME.zip
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(ckpt)
print('Use ckpt in {}'.format(ckpt_path))

# Alternatively access the checkpoint at https://drive.google.com/file/d/11L4jYRgCBUzchWt35SOMc2v_dmgZdxat/view?usp=sharing

## Prepare data

In [ ]:
# Prepare Training Data
!rm -rf *.zip *.tar tfrecord/ val2017/
!mkdir tfrecord
!PYTHONPATH=".:$PYTHONPATH"  python dataset/create_coco_tfrecord.py \
      --image_dir="{/Path/To/Train/Images}" \
      --object_annotations_file="/Path/To/TrainAnnotation/instances_Images.json" \
      --output_file_prefix=tfrecord/testdev \
      --num_shards=15

In [ ]:
# Prepare Validation Data
!PYTHONPATH=".:$PYTHONPATH"  python dataset/create_coco_tfrecord.py \
      --image_dir="{/Path/To/Val/Images}" \
      --object_annotations_file="/Path/To/ValAnnotation/instances_ImagesVal.json" \
      --output_file_prefix=tfrecord/Valdev \
      --num_shards=15

In [ ]:
file_pattern = 'testdev-*-of-00015.tfrecord' # Update to match the number of shards for training set
file_patternVal = 'Valdev-*-of-00015.tfrecord' # Update to match the number of shards for valid set

images_per_epoch = 57 * len(tf.io.gfile.glob('tfrecord/' + file_pattern))
images_per_epoch = images_per_epoch // 8 * 8  # round to 64.

images_per_epochVal = 57 * len(tf.io.gfile.glob('tfrecord/' + file_patternVal))
images_per_epochVal = images_per_epochVal // 8 * 8  # round to 64.

print('images_per_epoch = {}'.format(images_per_epoch))
print('images_per_epochVal = {}'.format(images_per_epochVal))

In [ ]:
# generating train tfrecord is large, so we skip the execution here.
import os

!mkdir model_dir/
# key option: use --ckpt rather than --backbone_ckpt.
!python main.py --mode=train_and_eval \
    --training_file_pattern=tfrecord/{file_pattern} \
    --validation_file_pattern=tfrecord/{file_patternVal} \
    --model_name={MODEL} \
    --model_dir=model_dir/{MODEL}-finetune \
    --ckpt= {ckpt} \
    --train_batch_size=8 \
    --eval_batch_size=8 --eval_samples={images_per_epochVal}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=2  \
    --hparams="num_classes=2,max_instances_per_image=5,learning_rate=0.001,moving_average_decay=0,mixed_precision=true,use_augmix=True"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir model_dir/
# Notably, this is just a demo with almost zero accuracy due to very limited
# training steps, but we can see finetuning has smaller loss than training
# from scratch at the begining.